In [36]:
import pandas as pd

In [37]:
current = pd.read_csv('src/results/results_summary_demand=13_posts=original_allocation=current_year=2019.csv').mean()

In [38]:
summary = pd.read_csv('src/results/results_summary.csv')
summary = summary[summary['Scenario'] == 'withRRV']
summary = summary.groupby(['Demand Level', 'Resource Level']).mean().reset_index()

In [152]:
def find_best(demand, kpi):
    data = summary[summary['Demand Level'] == demand][['Resource Level', kpi]].set_index('Resource Level') - current[kpi]
    if kpi == 'Overall Survival':
        if len(data[data[kpi] >= 0]) == 0:
            return '> 124'
        best_res_level = data[data[kpi] >= 0].idxmin()[kpi]  
    else:
        if len(data[data[kpi] <= 0]) == 0:
            return '> 124'
        best_res_level = data[data[kpi] <= 0].idxmax()[kpi]  
    allocations = pd.read_csv(f'../results/demand_{demand}_withRRV_out.csv', names=range(24), index_col=0)
    ea = int(allocations.loc[best_res_level, 2])
    rrv = int(allocations.loc[best_res_level, 5])
    return f"{best_res_level}, ({ea}, {rrv})"

In [153]:
current

Trial                             5.500000
Percent Abandoned                 0.000339
Ambulance Utilisation             0.317497
RRV Utilisation                   0.409045
Mean Response Time                0.011667
Percent Response < 8              0.061027
Percent Response < 15             0.448023
Percent Response < 60             0.999983
Percent Response within Target    0.754018
Overall Survival                  0.754774
dtype: float64

In [154]:
demands = [13, 19, 34, 45]
kpis = ['Mean Response Time', 'Overall Survival', 'Percent Abandoned']
pd.DataFrame({
    kpi: [find_best(d, kpi) for d in demands] for kpi in kpis
}, index=demands)

,Mean Response Time,Overall Survival,Percent Abandoned
13,"60, (50, 30)","60, (50, 30)","80, (66, 42)"
19,"70, (60, 30)","78, (63, 45)","101, (79, 66)"
34,"88, (75, 39)","106, (83, 69)",> 124
45,"99, (85, 42)",> 124,> 124


In [164]:
round(current['Mean Response Time'] * (24 * 60), 1)

16.8

In [165]:
round(current["Overall Survival"] * 100, 2)

75.48

In [166]:
round(current['Percent Abandoned'] * 100, 4)

0.0339